# Locating and classifying the expanded ocod dataset

This notebook runs through the process of locating properties withing the OA/LSOA system and classifying properties into one of the 5 types and 'unknown'

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import pandas as pd
import numpy as np
import os
import re
import io
import zipfile
#from helper_functions import *
from locate_and_classify_helper_functions import *


In [4]:
print("load ONSPD")
postcode_district_lookup = load_postocde_district_lookup("/tf/empty_homes_data/" + "ONSPD_NOV_2021_UK.zip", "Data/ONSPD_NOV_2021_UK.csv")
print("load expanded ocod")
ocod_data =  pd.read_csv("/tf/empty_homes_data/OCOD_cleaned_expanded2.csv")
print("pre-process expanded ocod data")
ocod_data = preprocess_expandaded_ocod_data(ocod_data, postcode_district_lookup)
print("load and pre-process the Land Registry price paid dataset")
price_paid_df = load_and_process_pricepaid_data("/tf/empty_homes_data/price_paid_files/", postcode_district_lookup)
print("add in missing Local authority codes to the ocoda dataset")
ocod_data = add_missing_lads_ocod(ocod_data, price_paid_df)
print("load and pre-process the voa business ratings list dataset")
voa_businesses = load_voa_ratinglist('/tf/empty_homes_data/' +'uk-englandwales-ndr-2017-listentries-compiled-epoch-0029-baseline-csv.csv', postcode_district_lookup)


load ONSPD


/tf/empty_homes_london/locate_and_classify_helper_functions.py:87: DtypeWarning: Columns (18,31,40,45) have mixed types. Specify dtype option on import or set low_memory=False.
  postcode_district_lookup = pd.read_csv(f)[['pcds','oslaua','oa11','lsoa11', 'msoa11', 'ctry']]


load expanded ocod
pre-process expanded ocod data


/tf/empty_homes_london/locate_and_classify_helper_functions.py:107: FutureWarning: The default value of regex will change from True to False in a future version.
  ocod_data['postcode2'] = ocod_data['postcode'].str.lower().str.replace("\s", "")
/tf/empty_homes_london/locate_and_classify_helper_functions.py:112: FutureWarning: The default value of regex will change from True to False in a future version.
  ocod_data['street_number2'] = ocod_data['street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)


load and pre-process the Land Registry price paid dataset
add in missing Local authority codes to the ocoda dataset
load and pre-process the voa business ratings list dataset


/tf/empty_homes_london/locate_and_classify_helper_functions.py:458: DtypeWarning: Columns (1,2,6,12,13,17,22) have mixed types. Specify dtype option on import or set low_memory=False.
  voa_businesses =  pd.read_csv(file_path,


## Using price paid data to match names

The land registry does not use standardised LAD codes or names and 
the LAD names it uses appear to be wrong sometimes. I need to know the LADs so that I only try road matching within local authorities to minimise the chance of having the same road twice. To get around this I will use the substantially larger database of the price paid data to get all the land registry district names and match them to the onsp using the postcodes. This works as there are a large number of sales in each district most of them will have a postcode. There are cases where the wrong district or postcode is applied meaning a single district name can have two or more lad11cd's, to solve this I simply take the lad11cd with the largest number of counts.

The resulting OCOD data frame has a LAD11CD for each entry, and thus allows the road matching to work effectively

# Street and buildings to match lsoa

This section fills in missing lsoa11cd using knowledge of the LAD11cd and the streets within it. This takes data from price paid and voa

In [5]:
##
##This process is quite convoluted and there is certainly a more efficient and pythonic way
## however the order within each filling method is important to ensure that there are no duplicates
## as this causes the OCOD dataset to grow with duplicates
##
ocod_data = street_and_building_matching(ocod_data, price_paid_df, voa_businesses)

replace the missing lsoa using street matching
replace the missing lsoa using building matching


/tf/empty_homes_london/locate_and_classify_helper_functions.py:541: FutureWarning: The default value of regex will change from True to False in a future version.
  temp['paon'] = temp['paon'].str.replace(r"\d+", "").replace(r"and|\&|-|,", "", regex = True).replace(r"^ +| +$", r"", regex=True)
/tf/empty_homes_london/locate_and_classify_helper_functions.py:549: FutureWarning: The default value of regex will change from True to False in a future version.
  pp_temp['paon'] = pp_temp['paon'].str.replace(r"\d+", "").replace(r"and|\&|-|,", "", regex = True).replace(r"^ +| +$", r"", regex=True)
/tf/empty_homes_london/locate_and_classify_helper_functions.py:549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pp_temp['paon'] = pp_temp['paon'].str.replace(r"

insert newly ID'd LSOA and OA
update missing LSOA and OA for nested properties where at least one nested property has an OA or LSOA


## Matching at sub street level

Some streets are on the boundary of LSOA this section uses the street number to match to the nearest lsoa.

In [7]:
#This takes some time
ocod_data = substreet_matching(ocod_data, price_paid_df, voa_businesses)
#percent of dataset without lsoa
ocod_data['lsoa11cd'].isnull().sum()/ocod_data.shape[0]

E06000031


/tf/empty_homes_london/locate_and_classify_helper_functions.py:637: FutureWarning: The default value of regex will change from True to False in a future version.
  missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)
/tf/empty_homes_london/locate_and_classify_helper_functions.py:637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)
/tf/empty_homes_london/locate_and_classify_helper_functions.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

i= 0  expand time,0.001 filter time0.0 make_dataframe_time 0.001
broadway
E07000032
albionclose
colliergrove
gadfieldgrove
E09000001


/tf/empty_homes_london/locate_and_classify_helper_functions.py:637: FutureWarning: The default value of regex will change from True to False in a future version.
  missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)
/tf/empty_homes_london/locate_and_classify_helper_functions.py:637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)
/tf/empty_homes_london/locate_and_classify_helper_functions.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.002
londonwall
cheapside
woodstreet
colemanstreet
princestreet
queenstreet
moorgate
cannonstreet
laurencepountneylane
gutterlane


...

/tf/empty_homes_london/locate_and_classify_helper_functions.py:637: FutureWarning: The default value of regex will change from True to False in a future version.
  missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)
/tf/empty_homes_london/locate_and_classify_helper_functions.py:637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)


byerlane
thesquare
E07000166


/tf/empty_homes_london/locate_and_classify_helper_functions.py:663: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_lsoa_road['lsoa11cd'] = [street_number_to_lsoa(temp_road, int(missing_lsoa_road.iloc[missing_lsoa_row]['street_number2']))
/tf/empty_homes_london/locate_and_classify_helper_functions.py:637: FutureWarning: The default value of regex will change from True to False in a future version.
  missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)
/tf/empty_homes_london/locate_and_classify_helper_functions.py:637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.001
marketplace


0.08883985889333483

## Add in counts of businesses per oa and LSOA

In [9]:
#This function allows areas with no  businesses to automatically exclude business from the classification
ocod_data = counts_of_businesses_per_oa_lsoa(ocod_data, voa_businesses)


## What still doesn't have an LSOA?
what still doesn't have lsoa and what properties do they have?

In [19]:
pd.crosstab(ocod_data['postcode'].notnull(), ocod_data['lsoa_building'].notnull())

lsoa_building,False,True
postcode,,
False,49390,7231
True,81337,21754


In [ ]:
test = ocod_data

In [ ]:
#observations localised with lsoa and/or oa
pd.crosstab(test['lsoa11cd'].notnull(),  test['oa11cd'].notnull())/ocod_data.shape[0]

In [ ]:
#this is definately the problem then
pd.crosstab(test['lsoa_street'].notnull(),  test['lsoa_building'].notnull())

In [ ]:
test2 = test[test['lsoa11cd'].isnull()]
pd.crosstab(test2['property_address'].str.startswith('land') , test2['lsoa_street'].notnull())

In [ ]:
#this is definately the problem then
pd.crosstab(test['street_name'].notnull(),  test['lsoa11cd'].notnull())

In [ ]:
test[test['lsoa11cd'].isnull() & test['street_name'].notnull()].to_csv('/tf/empty_homes_data/delete_me.csv')

In [21]:
#95.5% of sets have only a single lsoa, when grouped by street, town, district and locality
#when grouped by only street and district, this number is still 90%
#excluding town the number is still 0.95% but dropping locality gives a match on 91%, therefore using locality is the key
temp = price_paid_df.groupby(['street', 'district', 'lsoa11cd']).size().reset_index().groupby(['street', 'district']).size()\
.reset_index().rename(columns = {0:'counts'})

#temp.groupby('counts').size()/temp.shape[0]


# VOA matching businesses

The below chunk matches addresses to known businesses

In [11]:
ocod_data = voa_address_match_all_data(ocod_data, voa_businesses)

E06000031
i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.001


/tf/empty_homes_london/locate_and_classify_helper_functions.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_multi['number_filter'] = filter_types


E07000032
i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.001


/tf/empty_homes_london/locate_and_classify_helper_functions.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_multi['number_filter'] = filter_types


E09000001
i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.001


...

E07000166
E06000061


/tf/empty_homes_london/locate_and_classify_helper_functions.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_multi['number_filter'] = filter_types


i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.001
E06000062


/tf/empty_homes_london/locate_and_classify_helper_functions.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_multi['number_filter'] = filter_types
/tf/empty_homes_london/locate_and_classify_helper_functions.py:338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_multi['number_filter'] = filter_types


i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.001
E07000112
i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.001
E06000053


In [ ]:
pd.crosstab(ocod_data['oa_busi_building'].notnull(), ocod_data['business_address'].notnull())

# Classify property type

This section classifies the the data into different property types. 

# Classification type 1

The land is classified by the rules below which search the address string or meta data using regex.
The classification is hierarchical with the first match being the classification type.
Therefore if a property is classified by rule three and rule 6, rule three will take precedent and the property would be classed as airspace

- Starts with land/plot (land)
- Parking spaces (carpark)
- Air space (airspace)
- Flats, penthouses. apartments (domestic)
- Address matched businesses (business)
- Keyword relating to business (business)
- Land with other words before it (land)
- Pubs (business)
- A business was match in the same building (business)
- Is in the same address as a building (business)
- No business in the OA (domestic)
- No business in the LSOA (domestic)

After classifying the properties, classes left unknown are completed using the properties that are classed from the same title number
This is possible as there are no conflicting property classes within a given title number. This shows the quality of the method

In [15]:
ocod_data = classification_type1(ocod_data)

/tf/empty_homes_london/locate_and_classify_helper_functions.py:784: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.drop('class', axis = 1, inplace = True)


In [8]:
multi_class_titles = ocod_data[~ocod_data['class'].isin(['unknown', 'airspace', 'carpark']) & (ocod_data['within_larger_title']==True)].groupby(['title_number', 'class']).\
size().reset_index().groupby('title_number').size().reset_index().rename(columns={0:'counts'})

#there are no within title-ids that have more than one class. This shows that this is a very accurate way of filling in missing class data
print(multi_class_titles[multi_class_titles['counts']>1])

multi_class_titles = multi_class_titles[multi_class_titles['counts']==1]
#multi_class_titles.groupby('counts').size()

ocod_data[ocod_data['title_number'].isin(multi_class_titles['title_number'])].groupby('class').size()
#[['street_number', 'street_name','property_address', "business_address"]]

     title_number  counts
10         126312       2
16         142155       2
17         146577       2
19         147442       2
20         148312       2
...           ...     ...
4287    WYK737596       2
4294    WYK792514       2
4299    WYK856042       2
4304     YEA16295       2
4320      YY38811       2

[669 rows x 2 columns]


class
business     8046
domestic    40157
land            9
dtype: int64

## Classification type 2

Classification type 2 only affects the properties of class 'unknown' in classification type 1.

These properties are assumed to beeither domestic or business.
They are heierarchically classified into domestic or 'unknown' using the following rules

- Street match == TRUE, Street name is known AND street number is known (domestic)
- Street match is FALSE AND street name is known (domestic)
- Building name is known (domestic)

All remaining addresses do not contain enough information to be classified and are classed as unknown

In [18]:
ocod_data = classification_type2(ocod_data)

/tf/empty_homes_london/locate_and_classify_helper_functions.py:818: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.drop('class2', axis = 1, inplace = True)


In [ ]:
#If there is a street match, and the property has a street and a street number OR a building name
#Then is is a domestic property

test = ocod_data[ocod_data['class2']=='unknown']
print(pd.crosstab((test['street_match']==True), (test['street_name'].notnull()==True) ))

ocod_data[(ocod_data['street_name'].isnull()==True) & (ocod_data['class2']=='unknown')].to_csv('/tf/empty_homes_data/delete_me.csv')

In [ ]:
ocod_data.groupby('class').size()

In [ ]:
pd.crosstab(ocod_data['unit_type'],(ocod_data['class2']=="domestic"))

## Contracting the dataset
Businesses, carparks and airpsace etc are classed as a single address independent of how many components they a made of.
This chunk strips down businesses that have been expanded back to a single address


In [46]:
ocod_data = contract_ocod_after_classification(ocod_data, class_type = 'class2', classes = ['domestic'] )


In [ ]:
ocod_data.groupby('class2').size()

In [ ]:
ocod_data.groupby('class2').size()/ocod_data.shape[0]

In [45]:
#non of the unknowns have a postcode. I guess this is obvious as if there is no matching VOA postcode you are classed as domestic
#pd.crosstab(ocod_data[ocod_data['class']=="unknown"].postcode.notnull(), ocod_data[ocod_data['class']=="unknown"].street_name.notnull())

In [46]:
pd.crosstab(ocod_data['tenure'], ocod_data['region'].str.lower())#.to_latex() #convert to copyable latex table

region,east anglia,east midlands,greater london,north,north west,south east,south west,wales,west midlands,yorks and humber
tenure,,,,,,,,,,
Freehold,3418,4680,21783,3304,16221,17722,5754,3395,5050,8460
Leasehold,434,1209,40133,882,5741,5496,1702,693,1858,2793


# Saving the enhanced expanded dataset

In [48]:
ocod_data.to_csv("/tf/empty_homes_data/enhanced_ocod_dataset.csv")

#Save the test set indices to create the ground truth
#this is commented out to avoid overwriting

#ocod_data.loc[ocod_data.title_number.isin(pd.read_csv("/tf/empty_homes_data/test_set_indices.csv")['title_number']) ,  
#              ['title_number','within_title_id','unit_type' ,'building_name', 'street_number', 'street_name','postcode' ,'property_address',  'lsoa11cd', 'class2']].to_csv('/tf/empty_homes_data/parsed_ground_truth_raw.csv')

93707

## creating the aggregated ocod dataset for sampling

This chunk outputs the data ready for analysis in R. The two files cover the base case and the case where nested and single properties are separated

In [55]:
ocod_data_lsoa = ocod_data
#ocod_data_lsoa['postcode2'] = ocod_data['postcode'].str.lower().str.replace("\s", "")

#ocod_data_lsoa = ocod_data.merge(postcode_district_lookup, 'left', left_on = "postcode2", right_on = "postcode2")

ocod_data_lsoa.groupby(['lad11cd', 'lsoa11cd', 'msoa11cd', 'class2']).size().reset_index().to_csv("/tf/empty_homes_data/ocod_lsoa.csv")
ocod_data_lsoa.groupby(['lad11cd', 'lsoa11cd', 'msoa11cd', 'class2', 'within_larger_title']).size().reset_index().to_csv("/tf/empty_homes_data/ocod_lsoa_by_nested_type.csv")



# Post creation analysis

In [18]:
pd.crosstab(ocod_data['class2'], ocod_data['region'].str.lower())#.to_latex() #convert to copyable latex table

region,east anglia,east midlands,greater london,north,north west,south east,south west,wales,west midlands,yorks and humber
class2,,,,,,,,,,
airspace,50,162,94,15,215,185,266,290,94,303
business,550,1437,15806,799,2523,4453,1424,670,1423,1337
carpark,27,36,1597,24,518,194,89,9,50,57
domestic,2510,3791,48017,2742,16976,14306,4829,2607,4112,8439
land,639,919,2639,733,2215,4808,1270,629,1316,1314
unknown,64,115,734,157,319,453,207,89,150,133


In [19]:
pd.crosstab(ocod_data['class2'], ocod_data['tenure'])#.to_latex() #convert to copyable latex table

tenure,Freehold,Leasehold
class2,,
airspace,7,1667
business,20948,9474
carpark,252,2349
domestic,60744,47585
land,14619,1863
unknown,1534,887


In [20]:
temp_df = ocod_data[['title_number', 'tenure', 'within_larger_title']].drop_duplicates()

#most of titles containing nested addresses are free hold by about 3/2
pd.crosstab(temp_df['tenure'], temp_df['within_larger_title'])


within_larger_title,False,True
tenure,,
Freehold,46557,4208
Leasehold,41562,1381


In [21]:
#The analysis is based on nested addresses being domestic
temp_df = ocod_data[['title_number', 'tenure', 'property_address']][ocod_data['within_larger_title']==True]
temp_df['is_flat'] = temp_df['property_address'].str.contains(r"(flat|apartment|penthouse|unit)", case = False)

#pd.crosstab(temp_df['tenure'], temp_df['within_larger_title'])

temp_df.groupby('tenure').size()

#Of nested addresses freehold is more common by 3/2 50k to 24k
#most of theproperties are not flats however flats dominate the leasehold section
#flats are 1/3 of nested addresses but make up almost 3/4 of the leashold nested addresses
#note this does not include items marked as units
pd.crosstab(temp_df.tenure, temp_df.is_flat)

<ipython-input-21-c3f690fb0170>:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  temp_df['is_flat'] = temp_df['property_address'].str.contains(r"(flat|apartment|penthouse|unit)", case = False)


is_flat,False,True
tenure,,
Freehold,43388,7113
Leasehold,5186,16455


## Largest nested addresses

In [5]:
#The largest nested address
ocod_data.within_title_id.max()
ocod_data[ocod_data.within_title_id==ocod_data.within_title_id.max()].reset_index()['property_address'][0]



'Ground to ninth Floor Flats being 101-114, 201-214, 301-314, 401-414, 501-514, 601-613 and 701-704 Alaska Building, 101-114, 201-214,301-314, 401-412, 501-506 and 601-605 Arizona Building, 101-114, 201-214, 301-314, 401-414, 501-514, 601-614, 701-708, 801-804, 901-903 California Building, 101-108,     201-208, 301-307, 401-408, 501-508, 601-608, 701-708, 801-808 and 901-903 Colorado Building, 1-4, 101-109, 201-210, 301-310, 401-410, 501-510 and 601-605 Dakota Building, 1-7, 101-108, 201-208, 301-308, 401-408, 501-506 and 601-604 Idaho Building, 102-112, 201-212, 301-312, 401-412, 501-508 and 601-604 Indiana Building, 1-15, 101-116, 201-216, 301-315, 401-416, 501-510 Montana Building, 101-108, 201-208, 301-308, 401-408, 501-506 and 601-604 Nebraska Building, 1-10, 101-110, 201-210, 301-310 and 402-403 Utah Building, 1-10 and 101-110 Boston Building, 1-6, 101-106, 201-206, 301-306, 401-408 and 501-507 Madison Building, Deals Gateway, London'

# Whole location and classification pipeline

This provides a quick method to ouput the enhanced and expanded ocod dataset

In [4]:
print("Load ONSPD")
postcode_district_lookup = load_postocde_district_lookup("/tf/empty_homes_data/" + "ONSPD_NOV_2021_UK.zip", "Data/ONSPD_NOV_2021_UK.csv")
print("Load expanded ocod")
ocod_data =  pd.read_csv("/tf/empty_homes_data/OCOD_cleaned_expanded2.csv")
print("Pre-process expanded ocod data")
ocod_data = preprocess_expandaded_ocod_data(ocod_data, postcode_district_lookup)
print("Load and pre-process the Land Registry price paid dataset")
price_paid_df = load_and_process_pricepaid_data("/tf/empty_homes_data/price_paid_files/", postcode_district_lookup)
print("Add in missing Local authority codes to the ocoda dataset")
ocod_data = add_missing_lads_ocod(ocod_data, price_paid_df)
print("Load and pre-process the voa business ratings list dataset")
voa_businesses = load_voa_ratinglist('/tf/empty_homes_data/' +'uk-englandwales-ndr-2017-listentries-compiled-epoch-0029-baseline-csv.csv', postcode_district_lookup)
print("Match street addresses and buildings")
ocod_data = street_and_building_matching(ocod_data, price_paid_df, voa_businesses)

#This takes some time
print('Sub-street matching, this takes some time')
ocod_data = substreet_matching(ocod_data, price_paid_df, voa_businesses)
print('Add in businesses per oa and lsoa')
ocod_data = counts_of_businesses_per_oa_lsoa(ocod_data, voa_businesses)

print('Identify businesses using address matching')
ocod_data = voa_address_match_all_data(ocod_data, voa_businesses)

print('Classification type 1')
ocod_data = classification_type1(ocod_data)
print('Classification type 2')
ocod_data = classification_type2(ocod_data)

print('Contract ocod dataset')
ocod_data = contract_ocod_after_classification(ocod_data, class_type = 'class2', classes = ['domestic'] )
ocod_data.to_csv("/tf/empty_homes_data/enhanced_ocod_dataset.csv")

Load ONSPD


/tf/empty_homes_london/locate_and_classify_helper_functions.py:87: DtypeWarning: Columns (18,31,40,45) have mixed types. Specify dtype option on import or set low_memory=False.
  postcode_district_lookup = pd.read_csv(f)[['pcds','oslaua','oa11','lsoa11', 'msoa11', 'ctry']]


Load expanded ocod
Pre-process expanded ocod data


/tf/empty_homes_london/locate_and_classify_helper_functions.py:107: FutureWarning: The default value of regex will change from True to False in a future version.
  ocod_data['postcode2'] = ocod_data['postcode'].str.lower().str.replace("\s", "")
/tf/empty_homes_london/locate_and_classify_helper_functions.py:112: FutureWarning: The default value of regex will change from True to False in a future version.
  ocod_data['street_number2'] = ocod_data['street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)


Load and pre-process the Land Registry price paid dataset
Add in missing Local authority codes to the ocoda dataset
Load and pre-process the voa business ratings list dataset


...

i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.001
E07000112
i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.001
E06000053
Classification type 1


/tf/empty_homes_london/locate_and_classify_helper_functions.py:784: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.drop('class', axis = 1, inplace = True)


Classification type 2


/tf/empty_homes_london/locate_and_classify_helper_functions.py:818: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.drop('class2', axis = 1, inplace = True)


Contract ocod dataset


In [38]:
ground_truth_df = pd.read_csv('/tf/empty_homes_data/Ground truth for offshore empties V2 includes comma space_labels.csv')


#I only need a small number of the columns to be able to calculate the F1 score
#Everything else just makes it confusing. 
#renaming is for consistancy
ground_truth_df = ground_truth_df.loc[ground_truth_df.loc[:,'result_type']=="span",[ 'result_type', 'label',
       'start', 'end', 'text', 'input:text', 'input:datapoint_id']].rename(
    columns = {'input:text':'property_address',
              'input:datapoint_id':'datapoint_id',
              'text':'label_text'})

In [39]:
gt_class

,title_number,truth,class2
0,28418,business,business
1,89036,business,domestic
2,106920,domestic,domestic
3,145212,business,business
4,167012,business,business
...,...,...,...
1001,YY69405,domestic,domestic
1002,YY69484,domestic,domestic
1003,YY92067,land,land
1004,YY9279,airspace,airspace


In [26]:
from sklearn import metrics

In [54]:
gt_class = pd.read_csv('/tf/empty_homes_data/parsed_ground_truth_complete.csv').loc[:, ['title_number', 'truth']].drop_duplicates().\
merge(ocod_data.loc[:, ['title_number', 'class2']].drop_duplicates(), how = 'left')
label_names = list(np.unique(gt_class.truth.to_list()))

performance_df = metrics.precision_recall_fscore_support(gt_class.truth.to_list(),
                                        gt_class['class2'].to_list(), 
                                        labels = label_names)

performance_df = pd.DataFrame(np.round_(np.transpose(performance_df),2), columns = ["precision", "recall", "fscore", "support"])
performance_df['class'] = list(np.unique(gt_class.truth.to_list()))
performance_df[['class',"precision", "recall", "fscore", "support"]]

,class,precision,recall,fscore,support
0,airspace,1.00,0.93,0.96,14.0
1,business,0.99,0.77,0.86,294.0
2,carpark,1.00,0.96,0.98,26.0
3,domestic,0.88,0.97,0.92,484.0
4,land,1.00,0.99,1.00,179.0
5,unknown,0.00,0.00,0.00,9.0


In [52]:
gt_class = pd.read_csv('/tf/empty_homes_data/parsed_ground_truth_complete.csv').loc[:, ['title_number', 'truth']].\
merge(ocod_data.loc[:, ['title_number', 'class2']].drop_duplicates(), how = 'left')
label_names = list(np.unique(gt_class.truth.to_list()))

performance_df = metrics.precision_recall_fscore_support(gt_class.truth.to_list(),
                                        gt_class['class2'].to_list(), 
                                        labels = label_names)

performance_df = pd.DataFrame(np.round_(np.transpose(performance_df),2), columns = ["precision", "recall", "fscore", "support"])
performance_df['class'] = list(np.unique(gt_class.truth.to_list()))
performance_df[['class',"precision", "recall", "fscore", "support"]]

,class,precision,recall,fscore,support
0,airspace,1.00,0.93,0.96,14.0
1,business,0.99,0.72,0.83,332.0
2,carpark,1.00,0.96,0.98,26.0
3,domestic,0.91,0.98,0.95,919.0
4,land,1.00,0.99,1.00,179.0
5,unknown,0.00,0.00,0.00,9.0


0.9285714285714286

# Future work


The below are primarly nice to have things and would not change the output or results in any significant way

- I could re-insert the original street number in to the address when contracting this would be better for addresses that had been expanded but shouldn't have been. But defintaley isn't very important

- I could clean up the functions to remove the 'setting on copy' warning
- Create a verbose flag such that the messages and print outs of the functions are suppressed

In [54]:
test = ocod_data.loc[ocod_data.title_number.isin(pd.read_csv("/tf/empty_homes_data/test_set_indices.csv")['title_number']) ,  
             ['title_number','within_title_id','unit_type' ,'building_name', 'street_number', 'street_name','postcode' ,'property_address',  'lsoa11cd', 'class2']]

test.groupby('class2').size()

class2
airspace      13
business     210
carpark       25
domestic    1005
land         178
unknown       26
dtype: int64

In [15]:
test['property_address'].str.contains("^car").sum()

1

In [21]:
test.loc[test['property_address'].str.contains("^car"),'property_address']

495    cargo shown edged red on the title plan. the r...
Name: property_address, dtype: object